In [1]:
import pandas as pd 
import numpy as np 

In [2]:
pd.read_html('https://bulbapedia.bulbagarden.net/wiki/EX_Crystal_Guardians_(TCG)#Set_lists')

HTTPError: HTTP Error 403: Forbidden

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Path to your ChromeDriver executable
chrome_driver_path = '/path/to/chromedriver'

# Set up the Selenium driver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# URL of the page to scrape
url = 'https://bulbapedia.bulbagarden.net/wiki/EX_Crystal_Guardians_(TCG)#Set_lists'

# Open the page in the browser
driver.get(url)

# Find all the tables on the page
tables = driver.find_elements(By.TAG_NAME, 'table')

# Extract the tables using pd.read_html
dfs = pd.read_html(driver.page_source)

# Iterate over the extracted tables and print them
for i, df in enumerate(dfs):
    print(f"Table {i+1}:")
    print(df)
    print()

# Quit the browser
driver.quit()


Table 1:
                       0               1                      2
0    ← EX Holon Phantoms  TCG expansions  EX Dragon Frontiers →
1    ← EX Holon Phantoms             NaN                    NaN
2         TCG expansions             NaN                    NaN
3  EX Dragon Frontiers →             NaN                    NaN

Table 2:
                     0
0  ← EX Holon Phantoms

Table 3:
                0
0  TCG expansions

Table 4:
                       0
0  EX Dragon Frontiers →

Table 5:
                                                    0  \
0                          EX Crystal Guardiansきせきの結晶   
1                          EX Crystal Guardiansきせきの結晶   
2   Cards in set English: 100Japanese: 75 Set numb...   
3                                                 NaN   
4                                                 NaN   
5                                        Cards in set   
6                                          Set number   
7                                        Re

                                             PCG Era
0  PCG-P Promotional cardsVenusaur/Charizard/Blas...



In [10]:
len(dfs)

46

In [23]:
cards = dfs[14]

In [27]:
from selenium.webdriver.chrome.options import Options


In [31]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up Chrome driver service
service = Service('/path/to/chromedriver')  # Replace with the actual path to chromedriver

# Start the Chrome webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load the webpage
driver.get('https://bulbapedia.bulbagarden.net/wiki/EX_Crystal_Guardians_(TCG)#Set_lists')  # Replace with the actual URL of the webpage

# Wait for the page to load (you may need to adjust the waiting time depending on the page)
driver.implicitly_wait(10)  # Wait for 10 seconds

# Find the card element using the provided HTML snippet
card_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td')  # Replace with the actual XPath to the card element


# Extract the type and rarity of the card
card_type = card_element.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/th[@class="title"]')  # Replace with the actual XPath to the card type
card_rarity = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div[6]/div[4]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]')  # Revised XPath for card rarity

# Get the tables from the webpage using pd.read_html
tables = pd.read_html(driver.page_source)

# Assuming the table you want to update is the first table on the page
table = tables[14]

# Add the type and rarity to the respective row in the table
table['Type'] = card_type.text
table['Rarity'] = card_rarity.text

# Print the updated table
print(table)

# Close the Chrome webdriver
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/th[@class="title"]"}
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x0000000109d21598 chromedriver + 4973976
1   chromedriver                        0x0000000109d18913 chromedriver + 4938003
2   chromedriver                        0x00000001098d5cd6 chromedriver + 470230
3   chromedriver                        0x00000001099192ff chromedriver + 746239
4   chromedriver                        0x0000000109919591 chromedriver + 746897
5   chromedriver                        0x000000010990cf76 chromedriver + 696182
6   chromedriver                        0x000000010993d9ed chromedriver + 895469
7   chromedriver                        0x000000010990ce51 chromedriver + 695889
8   chromedriver                        0x000000010993db7e chromedri

In [30]:
table

,No.,Card name,Type,Rarity
0,1/100,Banette,,
1,2/100,Blastoise δ,,
2,3/100,Camerupt,,
3,4/100,Charizard δ,,
4,5/100,Dugtrio,,
...,...,...,...,...
96,97/100,Shiftry ex,,
97,98/100,Swampert ex,,
98,99/100,Alakazam,,
99,100/100,Celebi,,
